# preprocess data
split the eeg into small chunks

In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import torch as t

In [2]:
OUT_DIR = './preprocessed/'
BASE_PATH = './hms-harmful-brain-activity-classification/'
DATA_PATH = './hms-harmful-brain-activity-classification/train_eegs/'

In [3]:
def preprocess(df):
    sample_rate = 200
    duration = 10_000
    for eeg_id, group in tqdm(df.groupby('eeg_id')):
        parquet_file = f'{DATA_PATH}{eeg_id}.parquet'
        eeg = pd.read_parquet(parquet_file)
        for _, row in group.iterrows():
            eeg_sub_id = row['eeg_sub_id']
            offset = int(row['eeg_label_offset_seconds'] * sample_rate)
            eeg = eeg.iloc[offset:offset + duration]
            eeg = eeg.ffill(axis=0).fillna(0)
            data = t.tensor(eeg.values).float()
            t.save(data, f'{OUT_DIR}{eeg_id}_{eeg_sub_id}.pt')

df = pd.read_csv(f'{BASE_PATH}train.csv')
preprocess(df)

  0%|          | 0/17089 [00:00<?, ?it/s]